In [3]:
import numpy as np
import pandas as pd
import time
import gzip
import csv
import json
import pandas as pd

In [9]:
DIR = 'C:/Users/vicsa/OneDrive/Documents/Personal/Master_DataScience_UBA/2do Cuatrimestre/DMEyF/data/datasets'

In [14]:
with gzip.open(f'{DIR}/competencia_02_crudo.csv.gz', 'rt') as f:
    data = pd.read_csv(f, sep=',', quotechar='"')

In [15]:
print(data.columns)

Index(['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip',
       'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad',
       'mrentabilidad_annual', 'mcomisiones',
       ...
       'Visa_madelantodolares', 'Visa_fultimo_cierre', 'Visa_mpagado',
       'Visa_mpagospesos', 'Visa_mpagosdolares', 'Visa_fechaalta',
       'Visa_mconsumototal', 'Visa_cconsumos', 'Visa_cadelantosefectivo',
       'Visa_mpagominimo'],
      dtype='object', length=154)


In [16]:
print(data.head())

   numero_de_cliente  foto_mes  active_quarter  cliente_vip  internet  \
0          249221109    201901               1            0         1   
1          249221468    201901               1            0         1   
2          249223005    201901               1            0         1   
3          249228180    201901               1            0         1   
4          249232117    201901               1            0         1   

   cliente_edad  cliente_antiguedad  mrentabilidad  mrentabilidad_annual  \
0            59                 276        7597.55              47433.58   
1            51                  13         738.16               -325.09   
2            46                 178        1014.31               9434.15   
3            64                 296        1028.33               6873.80   
4            77                 349       11617.61              43291.55   

   mcomisiones  ...  Visa_madelantodolares  Visa_fultimo_cierre  Visa_mpagado  \
0      5654.59  ...    

In [18]:
# Cargamos los datos
#print('Cargando datos...')
#data = pd.read_csv(f'{DIR}/competencia_02_crudo.csv', header=0, sep=' ', quotechar='"')

# Inicio del proceso
print('Procesando...')
start_time = time.time()

# Trabajamos solamente con las columnas necesarias
clientes = data[['numero_de_cliente', 'foto_mes']].copy(
).sort_values(['numero_de_cliente', 'foto_mes'])

# Convertimos foto mes a un periodo numérico secuencial
clientes['periodo0'] = clientes['foto_mes'].floordiv(
    100).mul(12) + clientes['foto_mes'].mod(100)

# Calculamos el último y anteúltimo periodo
ultimo = clientes['periodo0'].max()
anteultimo = ultimo - 1

# Creamos los leads de período según cliente
groupper = clientes.groupby('numero_de_cliente')['periodo0']
clientes['periodo1'] = groupper.shift(-1)
clientes['periodo2'] = groupper.shift(-2)

# Si el cliente tiene vacío periodo 1, es BAJA+1, si tiene vacío periodo 2, es BAJA+2
clientes['clase_ternaria'] = np.where(clientes['periodo0'] == ultimo, pd.NA,
                                      np.where(clientes['periodo1'].isna(), 'BAJA+1',
                                               np.where(clientes['periodo0'] == anteultimo, pd.NA,
                                                        np.where(clientes['periodo2'].isna(), 'BAJA+2', 'CONTINUA'))))

# Fin del proceso
print('Tiempo de procesamiento:', round(
    time.time() - start_time, 4), 'segundos.')

# Guardamos el dataset
print('Guardando datos...')
data.join(clientes['clase_ternaria']).to_csv(
    f'{DIR}/competencia_02.csv', index=False)

# Mostramos la distribución de clases
print('Distribución de clases:')
print(clientes['clase_ternaria'].value_counts(
    normalize=True).mul(100).round(2))

Procesando...
Tiempo de procesamiento: 1.49 segundos.
Guardando datos...
Distribución de clases:
clase_ternaria
CONTINUA    99.09
BAJA+1       0.46
BAJA+2       0.45
Name: proportion, dtype: float64
